In [8]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from magenta.models.music_vae import configs

import numpy as np
from magenta.models.music_vae.trained_model import TrainedModel # We need this class

def build_standalone_encoder(hparams, input_shape):
    """
    Builds a standard Keras model that replicates the custom BidirectionalLstmEncoder.
    
    Args:
        hparams: The same hparams object used by the original model.
        input_shape: The shape of a single input sequence (e.g., (96, 64)).
    """
    
    # 1. Define the input layer
    input_tensor = tf.keras.layers.Input(shape=input_shape, name='encoder_input')
    
    # 2. Re-create the stacked LSTM cells as Keras layers
    # The original code uses a shared cell for fw and bw. The modern equivalent
    # is to create one stack of LSTM layers and wrap it in Bidirectional.
    
    # Start with the input tensor
    x = input_tensor
    
    # Create the stack of LSTM layers
    # The `return_sequences=True` is crucial for all but the last layer in a stack.
    num_lstm_layers = len(hparams.enc_rnn_size)
    for i, layer_size in enumerate(hparams.enc_rnn_size):
        lstm_layer = tf.keras.layers.LSTM(
            units=layer_size, 
            return_sequences=False, # The last (and only) LSTM returns a single vector
            name='multi_rnn_cell/cell_0/lstm_cell' # CRITICAL: Match TF1 variable scope
        )
        
        bi_lstm = tf.keras.layers.Bidirectional(
            lstm_layer,
            name='bidirectional_rnn' # CRITICAL: Match TF1 variable scope
        )

        # To create the 'cell_0' scope, we wrap the Bidirectional layer in a tiny sub-model
        # This is the key to matching the checkpoint's variable names.
        bi_lstm_model = tf.keras.Model(inputs=x, outputs=bi_lstm(x), name=f'cell_{i}')
        x = bi_lstm_model(x)

    # 3. Replicate the flatten operation
    # The original code flattens, but since the last LSTM returns a single vector per direction,
    # the output of Bidirectional is already "flat" in the time dimension.
    
    # 4. Re-create the final Dense layers
    # This layer needs to be inside an 'encoder' scope, which we will achieve by wrapping the whole thing.
    mu_layer = tf.keras.layers.Dense(hparams.z_size, name='mu')
    
    # The final output of our model is just the embedding (mu)
    mu_output = mu_layer(x)
    
    # 5. Create and return the final Keras model
    # We name the final model 'encoder' to add the last required scope.
    encoder_model = tf.keras.Model(inputs=input_tensor, outputs=mu_output, name='encoder')
    
    print("--- Standalone Keras Encoder Built with FINAL Corrected Naming ---")
    encoder_model.summary()
    
    return encoder_model

# --- Example Usage ---

# You would get `your_hparams` from the `mel_2bar._config.hparams` object
mel_2bar_config = configs.CONFIG_MAP['cat-mel_2bar_big']

# Derive the input_shape from the config
# At present mel_2bar_config.data_converter.input_depth is 90 (for 88 piano keys + 2 possibly special tokens)
# and mel_2bar_config.hparams.max_seq_len is 32 (for 2 bars of 16 steps each)
encoder_input_shape = (mel_2bar_config.hparams.max_seq_len, mel_2bar_config.data_converter.input_depth)

standalone_encoder = build_standalone_encoder(mel_2bar_config.hparams, input_shape=encoder_input_shape)


--- Standalone Keras Encoder Built with FINAL Corrected Naming ---
Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 32, 90)]          0         
                                                                 
 cell_0 (Functional)         (None, 4096)              35045376  
                                                                 
 mu (Dense)                  (None, 512)               2097664   
                                                                 
Total params: 37,143,040
Trainable params: 37,143,040
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Use tensorflow.compat.v1 and disable V2 behavior

sess = mel_2bar._sess
print("Original model loaded successfully.")

print("\n--- Step 2: Building the new standalone TF2 Keras encoder ---")
encoder_input_shape = (mel_2bar_config.hparams.max_seq_len, mel_2bar_config.data_converter.input_depth)
standalone_encoder = build_standalone_encoder(mel_2bar_config.hparams, input_shape=encoder_input_shape)

print("\n--- Step 3: Starting weight transfer from TF1 session to TF2 model ---")
tf1_variables = sess.graph.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
tf1_variable_names = [v.name for v in tf1_variables]
tf1_variable_values = sess.run(tf1_variables)
tf1_weights_map = dict(zip(tf1_variable_names, tf1_variable_values))

# Iterate through the layers of our new Keras model to set their weights
for layer in standalone_encoder.layers:
    if layer.name == 'cell_0': # This is our Bidirectional sub-model
        for sub_layer in layer.layers:
            if isinstance(sub_layer, tf.keras.layers.Bidirectional):
                print(f"\nProcessing layer: {layer.name}/{sub_layer.name}")
                
                # Get the names for the combined kernel and bias from the TF1 checkpoint
                fw_kernel_name = 'encoder/cell_0/bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel:0'
                fw_bias_name = 'encoder/cell_0/bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias:0'
                bw_kernel_name = 'encoder/cell_0/bidirectional_rnn/bw/multi_rnn_cell/cell_0/lstm_cell/kernel:0'
                bw_bias_name = 'encoder/cell_0/bidirectional_rnn/bw/multi_rnn_cell/cell_0/lstm_cell/bias:0'
                
                try:
                    # --- KERNEL SPLITTING LOGIC ---
                    input_depth = mel_2bar_config.data_converter.input_depth
                    
                    # Get the combined kernels from the checkpoint
                    fw_kernel_combined = tf1_weights_map[fw_kernel_name]
                    bw_kernel_combined = tf1_weights_map[bw_kernel_name]
                    
                    # Split the forward kernel
                    fw_input_kernel = fw_kernel_combined[:input_depth, :]
                    fw_recurrent_kernel = fw_kernel_combined[input_depth:, :]
                    
                    # Split the backward kernel
                    bw_input_kernel = bw_kernel_combined[:input_depth, :]
                    bw_recurrent_kernel = bw_kernel_combined[input_depth:, :]
                    
                    # Get the biases
                    fw_bias = tf1_weights_map[fw_bias_name]
                    bw_bias = tf1_weights_map[bw_bias_name]
                    
                    # Assemble the list of 6 weights in the correct order for Keras
                    keras_weights = [
                        fw_input_kernel, fw_recurrent_kernel, fw_bias,
                        bw_input_kernel, bw_recurrent_kernel, bw_bias
                    ]
                    
                    # Set the 6 weights on the Bidirectional layer
                    sub_layer.set_weights(keras_weights)
                    print(f"  - Successfully split kernels and transferred 6 weights for Bidirectional LSTM.")

                except KeyError as e:
                    print(f"  - FATAL ERROR: Could not find weight {e}!")
                except ValueError as e:
                    print(f"  - FATAL ERROR during set_weights: {e}")

    if layer.name == 'mu' and isinstance(layer, tf.keras.layers.Dense):
        print(f"\nProcessing layer: {layer.name}")
        kernel_name = f'encoder/mu/kernel:0'
        bias_name = f'encoder/mu/bias:0'
        try:
            layer.set_weights([
                tf1_weights_map[kernel_name],
                tf1_weights_map[bias_name]
            ])
            print(f"  - Successfully transferred weights for Dense layer 'mu'.")
        except KeyError as e:
            print(f"  - FATAL ERROR: Could not find weight {e} for 'mu' layer!")



print("\n--- Step 4: Weight transfer complete. Saving the new Keras model. ---")

output_keras_model_path = 'models/music_vae_encoder_keras'
tf.keras.models.save_model(standalone_encoder, output_keras_model_path)

print(f"\nSuccessfully saved the new Keras model to: '{output_keras_model_path}'")


Original model loaded successfully.

--- Step 2: Building the new standalone TF2 Keras encoder ---
--- Standalone Keras Encoder Built with FINAL Corrected Naming ---
Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 32, 90)]          0         
                                                                 
 cell_0 (Functional)         (None, 4096)              35045376  
                                                                 
 mu (Dense)                  (None, 512)               2097664   
                                                                 
Total params: 37,143,040
Trainable params: 37,143,040
Non-trainable params: 0
_________________________________________________________________

--- Step 3: Starting weight transfer from TF1 session to TF2 model ---

Processing layer: cell_0/bidirectional_rnn
  - Successfully split kernel

In [ ]:
# Use tensorflow.compat.v1 and disable V2 behavior for the original model
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

# ==============================================================================
# 1. SETUP & MODEL LOADING
# ==============================================================================

print("--- Step 1: Loading original TF1-style MusicVAE model ---")

# Use a batch size of 1 for easier comparison
BATCH_SIZE = 1
mel_2bar = TrainedModel(mel_2bar_config, batch_size=BATCH_SIZE, checkpoint_dir_or_path=checkpoint_path)
sess = mel_2bar._sess
print("Original model loaded.")

print("\n--- Step 2: Loading the new standalone Keras encoder ---")
keras_model_path = 'model/music_vae_encoder_keras'
# We can load the Keras model using the modern API
standalone_encoder = tf.keras.models.load_model(keras_model_path)
print("New Keras model loaded.")


# ==============================================================================
# 2. INFERENCE & COMPARISON
# ==============================================================================

print("\n--- Step 3: Generating a random input tensor ---")
# Get the expected input shape from the config
seq_len = mel_2bar_config.hparams.max_seq_len
input_depth = mel_2bar_config.data_converter.input_depth
input_shape = (BATCH_SIZE, seq_len, input_depth)

# Create a random input tensor
random_input = np.random.rand(*input_shape).astype(np.float32)
print(f"Generated random input with shape: {random_input.shape}")

print("\n--- Step 4: Running inference on both models ---")

# --- Get embedding from the ORIGINAL TF1 model ---
# We need to feed the input tensor and the sequence length to the session
feed_dict = {
    mel_2bar._inputs: random_input,
    # For a full-length random input, the length is the max sequence length
    mel_2bar._inputs_length: [seq_len] * BATCH_SIZE
}
# `_mu` is the tensor that holds the embedding (the mean of the latent distribution)
original_embedding = sess.run(mel_2bar._mu, feed_dict)


# --- Get embedding from the NEW Keras model ---
# The Keras model's predict method is much simpler
keras_embedding = standalone_encoder.predict(random_input)


# ==============================================================================
# 3. VERIFICATION
# ==============================================================================

print("\n--- Step 5: Comparing the outputs ---")
print(f"Original model's embedding (first 5 values): {original_embedding[0, :5]}")
print(f"New Keras model's embedding (first 5 values):  {keras_embedding[0, :5]}")

# Compare the two embeddings for near-perfect equality
# np.allclose is essential for comparing floating-point numbers
are_close = np.allclose(original_embedding, keras_embedding, atol=1e-6)

print("\n--- VERIFICATION RESULT ---")
if are_close:
    print("✅ SUCCESS: The embeddings from both models are identical!")
    # Calculate the absolute difference for confirmation
    abs_diff = np.mean(np.abs(original_embedding - keras_embedding))
    print(f"   (Mean absolute difference: {abs_diff:.10f})")
else:
    print("❌ FAILURE: The embeddings do not match.")
    # Calculate the absolute difference to see how far off they are
    abs_diff = np.mean(np.abs(original_embedding - keras_embedding))
    print(f"   (Mean absolute difference: {abs_diff})")

